In [1]:
import os
from BH.feature_functions import *

GPU_NUM = "0"
num_epochs = 500
batch_size = 128

use_ppath = False
column_info = "original"

graph_deg = 5
num_layers = graph_deg
num_features = 256
feature_list = {
    'constant':    (False, constant_feature),
    'column':      (False, column_indicator),
    'norm_column': (True, normalized_column_indicator), #   Feauture
    'norm_column_rev': (False, normalized_column_rev_indicator),
}
connected = True
UPTO = False

direction = "222"

shape_indicator = {
    'all_with_all_row_connectedness_criterion': (False, ),
    'all_with_inductive_connectedness_criterion': (True, ),
    '2row_less': (False, ),
    '2row_less_with_all_row_connectedness_criterion': (False, ),
    '2row_less_with_inductive_connectedness_criterion': (False, ),
    '3row_less_with_all_row_connectedness_criterion': (False, ),
    '3row_less_with_inductive_connectedness_criterion': (False, ),
}

shape= {
    'all': (False, ),# 3개다 
    '2row_less': (True, ),
    '3row_less': (False, ),
}
filter_indicator = {
    'with_all_row_connectedness_criterion':(False,), 
    'with_inductive_connectedness_criterion':(False,),
}


use_pretrained_weights = False
save_trained_weights = True

step_size = 0.0001
train_fraction = .8

DIR_PATH = f'/Data/Ptab/n={graph_deg}'
MODEL_DIR = './trained_models'
MODEL_FILE = os.path.join(MODEL_DIR, f'parameters_{graph_deg}_{num_layers}_{num_features}')

for key in shape_indicator:
    if shape_indicator[key][0] == True:
        DIR_PATH += f'_{key}'
        MODEL_FILE += f'_{key}'


if connected == True:
    DIR_PATH += "_connected"
    MODEL_FILE += "_connected"
elif connected == False:
    DIR_PATH += "_disconnected"
    MODEL_FILE += "_disconnected"
if UPTO == True:
    DIR_PATH += "_UPTO"
    MODEL_FILE += "_UPTO"
if not column_info == "original":
    DIR_PATH += column_info
    MODEL_FILE += column_info
if use_ppath:
    DIR_PATH += "_ppath"    
    MODEL_FILE += "_ppath"
    
for key in feature_list.keys():
    if feature_list[key][0] == True:
        MODEL_FILE += f'_{key}'
MODEL_FILE += f'_{direction}'
MODEL_FILE += '.pickle'



<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
import functools
import enum
import os
from BH.data_loader import *
# from BH.generate_data import *
#from BH.data_loader import *
#from BH.generate_data import *
# from training_info import *

import pickle

from CustomDataset import *
from GCN_model import *

from torch_geometric.loader import DataLoader


os.environ["CUDA_VISIBLE_DEVICES"] = GPU_NUM
device = "cuda:" + GPU_NUM


print("Loading input data...")
full_dataset, train_dataset, test_dataset = load_input_data(DIR_PATH)

node_dim = num_features
edge_dim = 8
graph_deg = graph_deg
depth = num_layers

test_dataset = CustomDataset(test_dataset)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

train_dataset = CustomDataset(train_dataset)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = pastGCN().to(device)
if use_pretrained_weights == True:
    try:
        with open(MODEL_FILE, 'rb') as f:
          model, max_accuracy, min_loss = pickle.load(f)
          model.to(device)
    except:
        print("There is no trained model")
        use_pretrained_weights = False
if use_pretrained_weights == False:
    model = GCN_multi_conv(graph_deg, depth, node_dim, direction).to(device)
    max_accuracy = 0
    min_loss = 100

# data = batch.to(device)
# torch.nn.init.xavier_normal(model)
# loss_function = torch.nn.CrossEntropyLoss()
loss_function = torch.nn.NLLLoss()
# loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=step_size, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                        lr_lambda=lambda epoch: 0.993 ** epoch,
                                        last_epoch=-1,
                                        verbose=False)

/root/anaconda3/envs/mpnn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading input data...
Generating data from the directory /Data/Ptab/n=5_all_with_inductive_connectedness_criterion_connected


In [7]:
outputs.shape

torch.Size([62, 6])

In [3]:
for epoch in range(num_epochs):
    # Training phase
    model.train()
    for batch in train_loader:
        batch.to(device)
        optimizer.zero_grad()
        out = model(batch)
        log_probs = torch.log(torch.clamp(out, min=1e-9))
#         log_probs = torch.log(out)
        loss = loss_function(log_probs, batch.y)
        loss.backward()
        # Apply gradient clipping
    
        optimizer.step()
    scheduler.step()
    print(loss)
    
    # Evaluation phase
    model.eval()
    correct = 0
    total = 0
#     with torch.no_grad():
#         for batch in test_loader:
#             batch.to(device)
#             outputs = model(batch)
# #             _,predicted = torch.max(outputs.data, 1)
#             predicted = outputs
#             total += batch.y.size(0)
# #             correct += (predicted == batch.y).sum().item()
#             correct += ((predicted - batch.y)**2<0.1).sum().item()
    with torch.no_grad():  # Disable gradient calculation
        total = 0
        correct = 0
        for batch in test_loader:
            # Move batch to the appropriate device (GPU or CPU)
            batch.to(device)

            # Forward pass: get model predictions
            outputs = model(batch)

            # Predicted classes are the indices of the max logits along the class dimension
            _, predicted = torch.max(outputs.data, 1)  # Shape: (batch_size)

            # Increment the total number of samples
            total += batch.y.size(0)

            # Compare predicted labels with ground truth labels
            correct += (predicted == batch.y).sum().item()  # Count the number of correct predictions

    # Compute the accuracy as a percentage
    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')
    # Compute accuracy
    accuracy = correct / total
    loss = float(loss.item())

    print("Epoch [{}/{}], Accuracy: {:.2%}, Loss: {:.15f}".format(epoch + 1, num_epochs, accuracy, loss))

    if (accuracy > max_accuracy or (accuracy == max_accuracy and loss < min_loss)) and save_trained_weights:
        max_accuracy = accuracy
        min_loss = loss
        with open(MODEL_FILE, 'wb') as f:
            pickle.dump((model, max_accuracy, min_loss), f)

/root/Min/Ptab/CustomDataset.py:18: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484809662/work/torch/csrc/utils/tensor_new.cpp:201.)
  edge_index = torch.tensor([self.rows[idx], self.cols[idx]], dtype=torch.long)


tensor(1.1003, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 59.68%
Epoch [1/500], Accuracy: 59.68%, Loss: 1.100312471389771
tensor(0.7472, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 59.68%
Epoch [2/500], Accuracy: 59.68%, Loss: 0.747235953807831
tensor(0.7064, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 61.29%
Epoch [3/500], Accuracy: 61.29%, Loss: 0.706436038017273
tensor(0.5544, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 58.06%
Epoch [4/500], Accuracy: 58.06%, Loss: 0.554357826709747
tensor(0.5735, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 59.68%
Epoch [5/500], Accuracy: 59.68%, Loss: 0.573463201522827
tensor(0.4666, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 74.19%
Epoch [6/500], Accuracy: 74.19%, Loss: 0.466624110937119
tensor(0.4926, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 80.65%
Epoch [7/500], Accuracy: 80.65%, Loss: 0.492577195167542
tensor(0.4376, device='cuda:0', grad_fn=<NllLossBackward0>)
Ac

tensor(0.1356, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 90.32%
Epoch [63/500], Accuracy: 90.32%, Loss: 0.135574936866760
tensor(0.1198, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 90.32%
Epoch [64/500], Accuracy: 90.32%, Loss: 0.119762927293777
tensor(0.1279, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 93.55%
Epoch [65/500], Accuracy: 93.55%, Loss: 0.127902463078499
tensor(0.1170, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 90.32%
Epoch [66/500], Accuracy: 90.32%, Loss: 0.117032609879971
tensor(0.1477, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 88.71%
Epoch [67/500], Accuracy: 88.71%, Loss: 0.147722601890564
tensor(0.1265, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 93.55%
Epoch [68/500], Accuracy: 93.55%, Loss: 0.126473873853683
tensor(0.1066, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 90.32%
Epoch [69/500], Accuracy: 90.32%, Loss: 0.106600463390350
tensor(0.1128, device='cuda:0', grad_fn=<NllLossBackwar

tensor(0.0588, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 90.32%
Epoch [125/500], Accuracy: 90.32%, Loss: 0.058807358145714
tensor(0.0522, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 93.55%
Epoch [126/500], Accuracy: 93.55%, Loss: 0.052249163389206
tensor(0.0516, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 90.32%
Epoch [127/500], Accuracy: 90.32%, Loss: 0.051633700728416
tensor(0.0699, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 88.71%
Epoch [128/500], Accuracy: 88.71%, Loss: 0.069923855364323
tensor(0.0616, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 93.55%
Epoch [129/500], Accuracy: 93.55%, Loss: 0.061638373881578
tensor(0.0570, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 90.32%
Epoch [130/500], Accuracy: 90.32%, Loss: 0.057033628225327
tensor(0.0440, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 88.71%
Epoch [131/500], Accuracy: 88.71%, Loss: 0.044035617262125
tensor(0.0477, device='cuda:0', grad_fn=<NllLoss

tensor(0.0228, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [187/500], Accuracy: 91.94%, Loss: 0.022797685116529
tensor(0.0453, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [188/500], Accuracy: 91.94%, Loss: 0.045258514583111
tensor(0.0248, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 90.32%
Epoch [189/500], Accuracy: 90.32%, Loss: 0.024802479892969
tensor(0.0275, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 90.32%
Epoch [190/500], Accuracy: 90.32%, Loss: 0.027525475248694
tensor(0.0367, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 90.32%
Epoch [191/500], Accuracy: 90.32%, Loss: 0.036710597574711
tensor(0.0374, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [192/500], Accuracy: 91.94%, Loss: 0.037411700934172
tensor(0.0400, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 90.32%
Epoch [193/500], Accuracy: 90.32%, Loss: 0.040040664374828
tensor(0.0388, device='cuda:0', grad_fn=<NllLoss

tensor(0.0263, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [249/500], Accuracy: 91.94%, Loss: 0.026260120794177
tensor(0.0227, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [250/500], Accuracy: 91.94%, Loss: 0.022713765501976
tensor(0.0221, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [251/500], Accuracy: 91.94%, Loss: 0.022142825648189
tensor(0.0213, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [252/500], Accuracy: 91.94%, Loss: 0.021293610334396
tensor(0.0305, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [253/500], Accuracy: 91.94%, Loss: 0.030538380146027
tensor(0.0247, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [254/500], Accuracy: 91.94%, Loss: 0.024705646559596
tensor(0.0214, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [255/500], Accuracy: 91.94%, Loss: 0.021432319656014
tensor(0.0335, device='cuda:0', grad_fn=<NllLoss

tensor(0.0160, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [311/500], Accuracy: 91.94%, Loss: 0.016008308157325
tensor(0.0165, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [312/500], Accuracy: 91.94%, Loss: 0.016508392989635
tensor(0.0212, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [313/500], Accuracy: 91.94%, Loss: 0.021207379177213
tensor(0.0252, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [314/500], Accuracy: 91.94%, Loss: 0.025166671723127
tensor(0.0192, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [315/500], Accuracy: 91.94%, Loss: 0.019222628325224
tensor(0.0201, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [316/500], Accuracy: 91.94%, Loss: 0.020145732909441
tensor(0.0154, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [317/500], Accuracy: 91.94%, Loss: 0.015397429466248
tensor(0.0178, device='cuda:0', grad_fn=<NllLoss

tensor(0.0207, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [373/500], Accuracy: 91.94%, Loss: 0.020698441192508
tensor(0.0145, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [374/500], Accuracy: 91.94%, Loss: 0.014487257227302
tensor(0.0176, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [375/500], Accuracy: 91.94%, Loss: 0.017615336924791
tensor(0.0164, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [376/500], Accuracy: 91.94%, Loss: 0.016398210078478
tensor(0.0184, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [377/500], Accuracy: 91.94%, Loss: 0.018425341695547
tensor(0.0166, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [378/500], Accuracy: 91.94%, Loss: 0.016629109159112
tensor(0.0194, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [379/500], Accuracy: 91.94%, Loss: 0.019392281770706
tensor(0.0222, device='cuda:0', grad_fn=<NllLoss

tensor(0.0135, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [435/500], Accuracy: 91.94%, Loss: 0.013503704220057
tensor(0.0187, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [436/500], Accuracy: 91.94%, Loss: 0.018712928518653
tensor(0.0191, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [437/500], Accuracy: 91.94%, Loss: 0.019089566543698
tensor(0.0186, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [438/500], Accuracy: 91.94%, Loss: 0.018598051741719
tensor(0.0190, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [439/500], Accuracy: 91.94%, Loss: 0.018984204158187
tensor(0.0203, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [440/500], Accuracy: 91.94%, Loss: 0.020290022715926
tensor(0.0146, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [441/500], Accuracy: 91.94%, Loss: 0.014623906463385
tensor(0.0143, device='cuda:0', grad_fn=<NllLoss

tensor(0.0135, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [497/500], Accuracy: 91.94%, Loss: 0.013533920980990
tensor(0.0171, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [498/500], Accuracy: 91.94%, Loss: 0.017081575468183
tensor(0.0149, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [499/500], Accuracy: 91.94%, Loss: 0.014914134517312
tensor(0.0155, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [500/500], Accuracy: 91.94%, Loss: 0.015534738078713


In [4]:
for batch in train_loader:
    print(batch)
    batch.to(device)
    out = model(batch)
    out_single = model.GCN_single(batch)
    break

DataBatch(x=[695, 1], edge_index=[2, 2085], y=[128], edge_types=[2085, 1], batch=[695], ptr=[129])


In [5]:
loss = loss_function(torch.log(out), batch.y)